In [34]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

In [36]:
# URL of the website
URL = "https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile"
# Set up the Chrome driver
driver = webdriver.Chrome()
driver.get(URL)
driver.maximize_window()
time.sleep(5)    # Wait for the page to load

In [37]:
# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name[i].text,
                    "Bus_Type": bus_type[i].text,
                    "Departing_Time": departing_time[i].text,
                    "Duration": duration[i].text,
                    "Reaching_Time": reaching_time[i].text,
                    "Star_Rating": star_rating[i].text if i < len(star_rating) else '0',
                    "Price": price[i].text,
                    "Seat_Availability": seat_availability[i].text if i < len(seat_availability) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)


        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('himachal_bus_details.csv', index=False)

# Close the driver
driver.quit()

In [38]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 538,Ordinary 3+2 Non AC Seater,18:31,10h 59m,05:30,4.0,INR 507,27 Seats available
1,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 93,Himsuta AC Seater Volvo/Scania 2+2,19:35,09h 10m,04:45,3.8,INR 892,14 Seats available
2,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 301,Ordinary 3+2 Non AC Seater,19:43,09h 47m,05:30,4.2,INR 507,26 Seats available
3,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 556,Ordinary 3+2 Non AC Seater,20:10,09h 35m,05:45,4.5,INR 507,27 Seats available
4,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...
404,Delhi to Sarkaghat,https://www.redbus.in/bus-tickets/delhi-to-sar...,HRTC - 534,Ordinary 3+2 Non AC Seater,18:05,11h 15m,05:20,4.5,INR 612,37 Seats available
405,Delhi to Sarkaghat,https://www.redbus.in/bus-tickets/delhi-to-sar...,HRTC - 541,Ordinary 3+2 Non AC Seater,19:14,08h 56m,04:10,4.6,INR 575,31 Seats available
406,Delhi to Sarkaghat,https://www.redbus.in/bus-tickets/delhi-to-sar...,HRTC - 549,A/C Executive (2+3),19:25,11h 20m,06:45,2.7,INR 701,33 Seats available
407,Delhi to Sarkaghat,https://www.redbus.in/bus-tickets/delhi-to-sar...,HRTC - 557,Ordinary 3+2 Non AC Seater,20:15,10h 55m,07:10,3.9,INR 575,31 Seats available
